# About

This notebook show how to train YOLOv5 object detector using custom data. For this purpose I created [Dataset](https://www.kaggle.com/datasets/maxkav/yolov5-game-dataset-for-darknet-framework?sort=recent-comments&select=notes.json) of images and labels.

Steps of all the process:
1. Collect lots of images.
2. Label images using labeling tool.
3. Train model and get weights file.
4. Initialize model with weights file & use it.

# Initialize constants

Set BASE_MODEL according to [Pretrained Checkpoints](https://github.com/ultralytics/yolov5/releases)

In [2]:
PROJECT_NAME = "yolov5_train"
BASE_MODEL = "yolov5m6.pt"
TRAIN_BATCH = 32
TRAIN_EPOCHS = 200
VAL_BATCH = 64

# Clone yolov5 repo

In [3]:
! git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16567, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 16567 (delta 23), reused 32 (delta 13), pack-reused 16522
Receiving objects: 100% (16567/16567), 15.02 MiB | 30.04 MiB/s, done.
Resolving deltas: 100% (11382/11382), done.


In [4]:
! pip install -qr yolov5/requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


# Import libraries

This notebook contains steps to train and evaluate yolov5 model with custom data from scratch. 

Steps to reproduce:
1. Collect lots of images.
2. Label images using labeling tool.
4. Train model and get weights file.
5. Initialize model with weights file & use it.

In [5]:
import torch
from yolov5 import utils
import torch
from IPython import display
from IPython.display import clear_output
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import io
import os
import cv2
import json
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

%matplotlib inline

# Convert data to yolov5 Pytorch format

Prepare data from Label Studio yolov5 darknet format to pytorch yolov5

In [7]:
IMAGES_PATH = "../input/croppedimagesarcpoints/cropped_images/images"
LABELS_PATH = "../input/croppedimagesarcpoints/cropped_images/labels"
NOTES_PATH = "../input/100-pie-charts-yolo/notes.json"

In [8]:
# Read labels
labels = os.listdir(LABELS_PATH)

# Split data
train, test = train_test_split(labels, test_size=0.8, shuffle=True)
valid, test = train_test_split(test, test_size=0.2)

print(f"train: {len(train)}; valid: {len(valid)}; test: {len(test)}")

train: 2000; valid: 6400; test: 1600


Make dirs for pytorch dataset format

In [9]:
os.makedirs("test/images")
os.makedirs("test/labels")
os.makedirs("train/images")
os.makedirs("train/labels")
os.makedirs("valid/images")
os.makedirs("valid/labels")

In [10]:
def move_files_to_dir(files, dirname):
    for label_filename in files:
        image_filename = f"{label_filename[:-4]}.png"
        shutil.copy(f"{IMAGES_PATH}/{image_filename}", f"{dirname}/images/{image_filename}")
        shutil.copy(f"{LABELS_PATH}/{label_filename}", f"{dirname}/labels/{label_filename}")

# Move splits to folders
move_files_to_dir(train, "train")
move_files_to_dir(test, "test")
move_files_to_dir(valid, "valid")

Convert yolov5-darknet to yolov5-pytorch description file

In [11]:
descr_darknet = json.load(open(NOTES_PATH))

train_path = "../train/images"
test_path = "../test/images"
valid_path = "../valid/images"

nc = len(descr_darknet["categories"])
names = [category["name"] for category in descr_darknet["categories"]]

print(
    f"train: {train_path}\n"
    f"test: {test_path}\n"
    f"val: {valid_path}\n\n"
    f"nc: {nc}\n"
    f"names: {names}",
)

train: ../train/images
test: ../test/images
val: ../valid/images

nc: 1
names: ['box']


In [12]:
with open("data.yaml", "w") as file:
    yaml.dump({
        "train": train_path,
        "test": test_path,
        "val": valid_path,
        "nc": nc,
        "names": [f'{name}' for name in names]
    }, stream=file, default_flow_style=None)

In [13]:
print("Now we are ready to train yolov5 model")
! ls 

Now we are ready to train yolov5 model
data.yaml  test  train	valid  yolov5


# Train yolov5

In [14]:
# Delete old results if exists
wildcard = f"{PROJECT_NAME}/feature_extraction*"
! rm -r $wildcard

rm: cannot remove 'yolov5_train/feature_extraction*': No such file or directory


In [15]:
torch.cuda.is_available()

True

In [16]:
! python yolov5/train.py --batch $TRAIN_BATCH --epochs $TRAIN_EPOCHS --data "data.yaml" --weights $BASE_MODEL --project $PROJECT_NAME --name 'feature_extraction' --cache --freeze 12

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-04-28 06:33:27.547803: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 06:33:27.547920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 06:33:27.692859: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train:

# Validation

In [45]:
# Delete old results
wildcard = f"{PROJECT_NAME}/validation_on_test_data*"
! rm -r $wildcard

In [15]:
WEIGHTS_BEST = f"{PROJECT_NAME}/feature_extraction/weights/best.pt"
! python yolov5/val.py --weights $WEIGHTS_BEST --batch $VAL_BATCH --data 'data.yaml' --task test --project $PROJECT_NAME --name 'validation_on_test_data' --augment

val: data=data.yaml, weights=['yolov5_train/feature_extraction/weights/best.pt'], batch_size=64, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5_train, name=validation_on_test_data, exist_ok=False, half=False, dnn=False
requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0'] not found, attempting AutoUpdate...
ERROR: Ignored the following versions that require a different python version: 10.0.0 Requires-Python >=3.8; 10.0.1 Requires-Python >=3.8; 10.1.0 Requires-Python >=3.8; 10.2.0 Requires-Python >=3.8; 10.3.0 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement pillow>=10.3.0 (from versions: 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7.0, 1.7.1, 1.7.2, 1.7.3, 1.7.4, 1.7.5, 1.7.6, 1.7.7, 1.7.8, 2.0.0, 2.1.0, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.3.2, 2.4.0, 2.5.0, 2.5.

# Test detection

In [17]:
# Delete old results
wildcard = f"{PROJECT_NAME}/detect_test*"
! rm -r $wildcard

rm: cannot remove 'yolov5_train/detect_test*': No such file or directory


In [18]:
! python yolov5/detect.py --weights "/kaggle/input/weights-edgearcpoints/weights_edge.pt" --conf 0.6 --source '/kaggle/input/croppedtestimages/cropped_images_test' --project $PROJECT_NAME --name 'detect_test' --augment --save-txt --line=3 

detect: weights=['/kaggle/input/weights-edgearcpoints/weights_edge.pt'], source=/kaggle/input/croppedtestimages/cropped_images_test, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=yolov5_train, name=detect_test, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-305-g4456c953 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)

Fusing layers... 
Model summary: 276 layers, 35248920 parameters, 0 gradients, 48.9 GFLOPs
image 1/10000 /kaggle/input/croppedtestimages/cropped_images_test/chart_20000.png: 640x640 11 boxs, 177.9ms
image 2/10000 /kaggle/input/croppedtestimages/cropped_images_test/chart_20001.png: 640x640 7 boxs, 41.3ms
image 3/10000 /kaggle/input/croppedtestima

## Creating a folder for inference

In [59]:
! mkdir validationImages

In [60]:
! cp /kaggle/input/think-cell-datathon/images/images/chart_{10..19999}.png /kaggle/working/validationImages/

In [19]:
! ls 

data.yaml  test  train	valid  yolov5  yolov5_train  yolov5m6.pt


In [66]:
! python yolov5/detect.py --weights $WEIGHTS_BEST --conf 0.6 --source 'validationImages' --project $PROJECT_NAME --name 'detect_test' --augment --save-txt --line=3 --max-det=1

detect: weights=['yolov5_train/feature_extraction/weights/best.pt'], source=validationImages, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=yolov5_train, name=detect_test, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0'] not found, attempting AutoUpdate...
ERROR: Ignored the following versions that require a different python version: 10.0.0 Requires-Python >=3.8; 10.0.1 Requires-Python >=3.8; 10.1.0 Requires-Python >=3.8; 10.2.0 Requires-Python >=3.8; 10.3.0 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement pillow>=10.3.0 (from versions: 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7.0, 1.7.1,

In [20]:
! cp /kaggle/working/yolov5_train/detect_test/chart_27512.png /kaggle/working

In [21]:
!tar -zcvf yolotest.tar.gz /kaggle/working/yolov5_train/detect_test

tar: Removing leading `/' from member names
/kaggle/working/yolov5_train/detect_test/
/kaggle/working/yolov5_train/detect_test/chart_27139.png
/kaggle/working/yolov5_train/detect_test/chart_22636.png
/kaggle/working/yolov5_train/detect_test/chart_24757.png
/kaggle/working/yolov5_train/detect_test/chart_24690.png
/kaggle/working/yolov5_train/detect_test/chart_26187.png
/kaggle/working/yolov5_train/detect_test/chart_21176.png
/kaggle/working/yolov5_train/detect_test/chart_28814.png
/kaggle/working/yolov5_train/detect_test/chart_29283.png
/kaggle/working/yolov5_train/detect_test/chart_20416.png
/kaggle/working/yolov5_train/detect_test/chart_24223.png
/kaggle/working/yolov5_train/detect_test/chart_27675.png
/kaggle/working/yolov5_train/detect_test/chart_25609.png
/kaggle/working/yolov5_train/detect_test/chart_23138.png
/kaggle/working/yolov5_train/detect_test/chart_29187.png
/kaggle/working/yolov5_train/detect_test/chart_24785.png
/kaggle/working/yolov5_train/detect_test/chart_27853.png
/k

In [17]:
def read_images(dirpath):
  images = []
  for img_filename in os.listdir(dirpath):
    images.append(mpimg.imread(f"{dirpath}/{img_filename}"))
  return images

In [18]:
def label_test_images(test_images_path, test_labels_path, classes):
  test_images = os.listdir(test_images_path)
  labeled_images = []

  for idx, test_image_filename in enumerate(test_images):
    image = mpimg.imread(f"{test_images_path}/{test_image_filename}")
    
    x_shape, y_shape = image.shape[1], image.shape[0]

    test_label_filename = f"{test_image_filename[:-4]}.txt"
    
    with open(f"{test_labels_path}/{test_label_filename}", "r") as f:
      lines = f.readlines()

      for line in lines:
        # Parse line
        box = line.split()
        class_idx = box[0]
        
        class_name = names[int(class_idx)]
        x_center, y_center, box_w, box_h = int(float(box[1])*x_shape), int(float(box[2])*y_shape), int(float(box[3])*x_shape), int(float(box[3])*y_shape)
        x1, y1, x2, y2 = x_center-int(box_w/2), y_center-int(box_h/2), x_center+int(box_w/2), y_center+int(box_h/2)

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
        cv2.putText(image, class_name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 3)

    labeled_images.append(image)

  return labeled_images

In [28]:
detect_path = f"{PROJECT_NAME}/detect_test"
test_images_path = f"test/images"
test_labels_path = f"test/labels"

detected_images = read_images(detect_path)
print(len(detected_images[1]))
test_labeled_images = label_test_images(test_images_path, test_labels_path, classes=names)
print(len(test_labeled_images[1]))
stacked_images = [np.hstack([detected_images[idx], test_labeled_images[idx]]) for idx in range(1)]

554
554


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 2, the array at index 0 has size 3 and the array at index 1 has size 4

In [ ]:
! ls

In [21]:
for image in stacked_images:
  fig = plt.figure(figsize=(40, 15))
  ax1 = fig.add_subplot(2,2,1)
  ax1.imshow(image)

NameError: name 'stacked_images' is not defined

# Save model

To save your model just download best.pt file from PROJECT_FOLDER -> feature_extraction (your best) -> weights -> best.pt

File best.pt will be used to load it in your project to predict.